Импортируем модули, которые нам понадобятся для решения задачи

In [2]:
pip install datasketch

In [3]:
pip install python-Levenshtein

In [4]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
import datasketch
from datasketch import MinHash, MinHashLSH
import Levenshtein

Загрузим наш датасет и выведем первые 4 значения

In [5]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [6]:
# printing the first 4 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
# number of rows and columns in the data frame

movies_data.shape

(4803, 24)

In [8]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [9]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [54]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [11]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [12]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Cosine Similarity

In [35]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)


In [36]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [37]:
print(similarity.shape)

(4803, 4803)


Дальше нужно будет ввести название любимого фильма, чтобы программа, впоследствие вывела рекомендуемые фильмы


In [38]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : catch me if you can


In [39]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [40]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Catch Me If You Can']


In [41]:
close_match = find_close_match[0]
print(close_match)

Catch Me If You Can


In [42]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

883


In [43]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.01268429563497391), (1, 0.026357002530503562), (2, 0.03984618904595524), (3, 0.06784991004941157), (4, 0.0), (5, 0.013139937207188056), (6, 0.0), (7, 0.0), (8, 0.025051664183472776), (9, 0.0), (10, 0.005432135182065453), (11, 0.0345970942071234), (12, 0.0), (13, 0.004452206741854051), (14, 0.022784118531214335), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.004614256112102113), (19, 0.03253658478004812), (20, 0.030705524070690642), (21, 0.0), (22, 0.029524051155481672), (23, 0.008352299578915838), (24, 0.05254710445159432), (25, 0.08249509199175924), (26, 0.0), (27, 0.004399482455089717), (28, 0.0048301125540836055), (29, 0.031883338273434654), (30, 0.01583279749058633), (31, 0.009492464289851968), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.004548294679611917), (40, 0.00511712010748202), (41, 0.0192713620416566), (42, 0.051502193425548265), (43, 0.00981834987900282), (44, 0.0), (45, 0.002760699069161695), (46, 0.009284927145877737), (47, 0.0), 

In [44]:
len(similarity_score)

4803

In [45]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(883, 1.0000000000000002), (1409, 0.1835958650211726), (351, 0.1724829199757023), (96, 0.1479475485564351), (1187, 0.14276698632698678), (628, 0.1420867850897968), (250, 0.1308327432428682), (1149, 0.12688786457764178), (3578, 0.12581523312210538), (2597, 0.12447026507534441), (521, 0.12322131552994621), (3014, 0.12269437500917499), (4366, 0.1204212195655), (1612, 0.11922945329331001), (316, 0.1179714791795152), (298, 0.11572083995099185), (1909, 0.11569365537971041), (652, 0.11540495513231742), (3496, 0.11427617139695484), (439, 0.11199694793492364), (3239, 0.11063866731279128), (1169, 0.10994815980427926), (1894, 0.10891529934457017), (2722, 0.10751854159743687), (1038, 0.10721084311262097), (2448, 0.10647729185318401), (1208, 0.10572123244487501), (1380, 0.10571445256890155), (1386, 0.10566751199273071), (1233, 0.1013450028141501), (1422, 0.10062081283472747), (4010, 0.0983762116185543), (1450, 0.09688177001285224), (524, 0.09576518960428221), (176, 0.0948633189929366), (287, 0.094

In [46]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Catch Me If You Can
2 . J. Edgar
3 . The Departed
4 . Inception
5 . Bridge of Spies
6 . Saving Private Ryan
7 . The Aviator
8 . American Hustle
9 . House Party 2
10 . Confidence
11 . The Terminal
12 . The Dead Zone
13 . Who Killed the Electric Car?
14 . Larry Crowne
15 . Gangs of New York
16 . The Wolf of Wall Street
17 . Legend
18 . Focus
19 . The Flower of Evil
20 . Shutter Island
21 . Stand by Me
22 . 42
23 . War
24 . Seven Psychopaths
25 . The Infiltrator
26 . Wall Street
27 . Jersey Boys
28 . The Man in the Iron Mask
29 . Saving Mr. Banks


Movie Recommendation Sytem

In [47]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<32):
    if(i!=1):
      print(i-1, '.',title_from_index)
    i+=1

 Enter your favourite movie name : catch me if you can
Movies suggested for you : 

1 . J. Edgar
2 . The Departed
3 . Inception
4 . Bridge of Spies
5 . Saving Private Ryan
6 . The Aviator
7 . American Hustle
8 . House Party 2
9 . Confidence
10 . The Terminal
11 . The Dead Zone
12 . Who Killed the Electric Car?
13 . Larry Crowne
14 . Gangs of New York
15 . The Wolf of Wall Street
16 . Legend
17 . Focus
18 . The Flower of Evil
19 . Shutter Island
20 . Stand by Me
21 . 42
22 . War
23 . Seven Psychopaths
24 . The Infiltrator
25 . Wall Street
26 . Jersey Boys
27 . The Man in the Iron Mask
28 . Saving Mr. Banks
29 . Get Rich or Die Tryin'
30 . The X Files: I Want to Believe


Теперь попробуем добавить дополнительные данные для фильмов и для дальнейшего сравнения.



In [53]:
combined_features_a = movies_data['genres'] + ' ' + \
                      movies_data['keywords'] + ' ' + \
                      movies_data['tagline'] + ' ' + \
                      movies_data['cast'] + ' ' + \
                      movies_data['vote_average'].astype(str) + ' ' + \
                      movies_data['release_date'].astype(str)


In [55]:
print(combined_features_a)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [57]:
feature_vectors_a = vectorizer.fit_transform(combined_features_a)

In [59]:
similarity_a = cosine_similarity(feature_vectors_a)

In [60]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : catch me if you can


In [64]:
similarity_score_a = list(enumerate(similarity_a[index_of_the_movie]))
print(similarity_score)

[(0, 0.022080378986794286), (1, 0.026995240087152636), (2, 0.04021941398798277), (3, 0.048231149952420574), (4, 0.0), (5, 0.013419776359656316), (6, 0.0), (7, 0.0), (8, 0.02664354470203608), (9, 0.0), (10, 0.00554762974082245), (11, 0.035140006429261995), (12, 0.0), (13, 0.004582067262112558), (14, 0.03342059731005105), (15, 0.0), (16, 0.017424077599388972), (17, 0.0), (18, 0.004695996906314992), (19, 0.0435334762653563), (20, 0.03152207902641922), (21, 0.007987472916864516), (22, 0.040926338439851745), (23, 0.016689150863324072), (24, 0.06158351546165297), (25, 0.08240747524503615), (26, 0.0), (27, 0.0044639850781450375), (28, 0.004985569703690153), (29, 0.05086120494289071), (30, 0.03400672245725035), (31, 0.009603105827379381), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.01676106387711509), (37, 0.0), (38, 0.0), (39, 0.013884846766593777), (40, 0.005276875465981239), (41, 0.0), (42, 0.05441616459366122), (43, 0.009876877749889027), (44, 0.0), (45, 0.0028068490993761047), (46,

In [65]:
sorted_similar_movies_a = sorted(similarity_score_a, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies_a)

[(883, 0.9999999999999997), (1409, 0.18822431833768852), (351, 0.16113519747388255), (1149, 0.14588189199494325), (3578, 0.14309995821628282), (2597, 0.14001001276743014), (176, 0.13535336663351857), (652, 0.1350899020710622), (96, 0.13128602108546517), (4366, 0.1296872100262367), (3014, 0.12782161650456123), (316, 0.1263344855295344), (287, 0.12521729926396916), (1169, 0.12488169041434728), (250, 0.12254833735405829), (298, 0.12245197451335027), (1909, 0.12132683737974186), (2448, 0.12019836691126898), (444, 0.12008028774682884), (1380, 0.11849069887506393), (3496, 0.118221904055302), (1894, 0.11187507557973547), (3239, 0.11147483106585686), (1208, 0.11076708304429393), (1386, 0.11037597001482778), (1038, 0.10920048468547018), (1450, 0.10863988615606929), (524, 0.10812162614506499), (497, 0.10721274393722674), (3526, 0.10349657565551601), (297, 0.10336946734530361), (1233, 0.103156121790452), (308, 0.10097789951739869), (1422, 0.1007908709843789), (4740, 0.10067345844968893), (1231, 0

In [66]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies_a:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Catch Me If You Can
2 . J. Edgar
3 . The Departed
4 . American Hustle
5 . House Party 2
6 . Confidence
7 . The Revenant
8 . Focus
9 . Inception
10 . Who Killed the Electric Car?
11 . The Dead Zone
12 . Gangs of New York
13 . Django Unchained
14 . 42
15 . The Aviator
16 . The Wolf of Wall Street
17 . Legend
18 . Wall Street
19 . Road to Perdition
20 . The Man in the Iron Mask
21 . The Flower of Evil
22 . War
23 . Stand by Me
24 . Jersey Boys
25 . Saving Mr. Banks
26 . The Infiltrator
27 . Punisher: War Zone
28 . Charlie Wilson's War
29 . Red Dragon


Видим, что вывод изменился.

In [75]:
combined_features_b = movies_data['genres'] + ' ' + \
                      movies_data['keywords'] + ' ' + \
                      movies_data['tagline'] + ' ' + \
                      movies_data['cast'] + ' ' + \
                      movies_data['vote_average'].astype(str) + ' ' + \
                      movies_data['release_date'].astype(str) +' ' + movies_data['director']

In [76]:
feature_vectors_b = vectorizer.fit_transform(combined_features_b)

In [77]:
similarity_b = cosine_similarity(feature_vectors_b)

In [78]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : catch me if you can


In [79]:
similarity_score_b = list(enumerate(similarity_b[index_of_the_movie]))
print(similarity_score)

[(0, 0.022080378986794286), (1, 0.026995240087152636), (2, 0.04021941398798277), (3, 0.048231149952420574), (4, 0.0), (5, 0.013419776359656316), (6, 0.0), (7, 0.0), (8, 0.02664354470203608), (9, 0.0), (10, 0.00554762974082245), (11, 0.035140006429261995), (12, 0.0), (13, 0.004582067262112558), (14, 0.03342059731005105), (15, 0.0), (16, 0.017424077599388972), (17, 0.0), (18, 0.004695996906314992), (19, 0.0435334762653563), (20, 0.03152207902641922), (21, 0.007987472916864516), (22, 0.040926338439851745), (23, 0.016689150863324072), (24, 0.06158351546165297), (25, 0.08240747524503615), (26, 0.0), (27, 0.0044639850781450375), (28, 0.004985569703690153), (29, 0.05086120494289071), (30, 0.03400672245725035), (31, 0.009603105827379381), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.01676106387711509), (37, 0.0), (38, 0.0), (39, 0.013884846766593777), (40, 0.005276875465981239), (41, 0.0), (42, 0.05441616459366122), (43, 0.009876877749889027), (44, 0.0), (45, 0.0028068490993761047), (46,

In [80]:
sorted_similar_movies_b = sorted(similarity_score_b, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies_b)

[(883, 0.9999999999999999), (1409, 0.17616194105904026), (351, 0.16553376433252268), (96, 0.1414684068783533), (1149, 0.13982734906596284), (316, 0.13893917323966684), (298, 0.1376555322343445), (1187, 0.13701808098388527), (628, 0.13612332592738616), (250, 0.13395934115149205), (2597, 0.13374131750121285), (652, 0.12642877671277714), (521, 0.11754766963180692), (3014, 0.1172612372278075), (3578, 0.11680011838830914), (287, 0.11583103571348476), (4366, 0.11542533792450235), (176, 0.11500177736644852), (1169, 0.11499516402856341), (1612, 0.1139090973223972), (444, 0.11365574212145933), (2448, 0.11056091143594442), (1380, 0.11035477980745918), (1909, 0.11029072997070294), (3496, 0.11023270004855755), (3239, 0.10662048130016792), (439, 0.10654844346743843), (1894, 0.10377038174631328), (2722, 0.10329281394816847), (1038, 0.10261352997678355), (1450, 0.1016560580712689), (1208, 0.10083739439438369), (1386, 0.10076746073920366), (524, 0.10049738735143302), (497, 0.09844176889458847), (1233,

In [81]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies_b:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Catch Me If You Can
2 . J. Edgar
3 . The Departed
4 . Inception
5 . American Hustle
6 . Gangs of New York
7 . The Wolf of Wall Street
8 . Bridge of Spies
9 . Saving Private Ryan
10 . The Aviator
11 . Confidence
12 . Focus
13 . The Terminal
14 . The Dead Zone
15 . House Party 2
16 . Django Unchained
17 . Who Killed the Electric Car?
18 . The Revenant
19 . 42
20 . Larry Crowne
21 . Road to Perdition
22 . Wall Street
23 . The Man in the Iron Mask
24 . Legend
25 . The Flower of Evil
26 . Stand by Me
27 . Shutter Island
28 . War
29 . Seven Psychopaths
